# Value at Risk (VaR)

## Introduction

VaR was developed in mid-1990s, in response to the various financial crisis, but the origins of the measures lie further back in time. According to Philippe Jorion, “VaR measures the worst expected loss over a given horizon under normal market conditions at a given level of confidence”.

This definition implies that it is necessary to choose two parameters, namely holding period and confidence level. Typically the confidence level lies in the range 90% to 99% such as 90%, 95%, 99%. Holding period may vary from a day to a year.

Suppose, an analyst says that the 1-day VaR of a portfolio is £1,000,000 with a 95% confidence level. It implies there is 95% chance that the maximum losses will not exceed £1,000,000 in a single day. In other words, there is only 5% chance that the portfolio losses on a particular day will be greater than £1,000,000.

The Variance-Covariance is a parametric method which assumes that the returns are normally distributed.

In [9]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from scipy.stats import norm

In [10]:
df = pdr.get_data_yahoo('JPM')

### 1) Calculate the Returns of the Closing Price

In [18]:
df['% Returns'] = df['Adj Close'].pct_change()
df['% Returns'].head(5)

Date
2010-01-04         NaN
2010-01-05    0.019370
2010-01-06    0.005495
2010-01-07    0.019808
2010-01-08   -0.002456
Name: % Returns, dtype: float64

### 2) Calculate the Mean of the Daily Returns

In [12]:
mu = np.mean(df['% Returns'])
mu

0.0006316385901777547

### 3) Calculate the Standard Deviation of the Returns

In [13]:
sigma = np.std(df['% Returns'])
sigma

0.015935320953654516

### 4) Calculate the VaR for 90, 95 & 99% Confidence Levels

Consider a portfolio of $P$ dollars, with a confidence level $c$. We are considering daily returns, with asset (or strategy) historical standard deviation $\sigma$ and mean $\mu$. Then the daily VaR, under the variance-covariance method for a single asset (or strategy) is calculated as:

$P−(P(\Phi^{-1}(1−c)+1))$

__Where:__ 

$\Phi$ is the inverse of the cumulative distribution function of a normal distribution

$\mu$ is the mean

$\sigma$ is the standard deviation

In [19]:
def value_at_risk(portfolio_value, confidence_interval, mu, sigma):
    """Calculates VaR using the Variance-Covariance Model."""
    alpha = norm.ppf(1 - confidence_interval, mu, sigma)
    
    return portfolio_value - (portfolio_value * (alpha + 1))

----

In [20]:
value_at_risk(portfolio_value=1000000, confidence_interval=0.9, mu=mu, sigma=sigma)

19790.29692543391

In [21]:
value_at_risk(portfolio_value=1000000, confidence_interval=0.95, mu=mu, sigma=sigma)

25579.631877076696

In [22]:
value_at_risk(portfolio_value=1000000, confidence_interval=0.99, mu=mu, sigma=sigma)

36439.46143251483